In [2]:
TICTACTOE_PROMPT_BASE_4O = """
You are playing tic-tac-toe. Your goal is to win the game. Your symbol is {}. We are currently on move number {}. Below, you’ll see the game history and a representation of the board. ou need to specify the number of the cell where you want to place your symbol. The cells are numbered from top to bottom, left to right: the first row has 1, 2, 3; the second row has 4, 5, 6; and the third row has 7, 8, 9. In your response, include a string with the reasoning you used to find the solution and the number corresponding to the cell where you want to place your symbol, like in the format below.

Game history: ###
{}
###

Board: ###
{}
###

Desired format: ###
Reasoning: ....

Cell Number: 

###
"""

In [1]:
import json
import sys
import os
import importlib
import re
from pydantic import BaseModel

In [3]:
TICTACTOE_PROMPT_BASE_O1 = """
You are playing tic-tac-toe. Your goal is to win the game. Your symbol is {}. We are currently on move number {}. Below, you’ll see the current situation and a representation of the board. You need to specify the number of the cell where you want to place your symbol. The cells are numbered from top to bottom, left to right: the first row has 1, 2, 3; the second row has 4, 5, 6; and the third row has 7, 8, 9. In your response, include only the number corresponding to the cell where you want to place your symbol (without any additional information).

Game history: ###
{}
###

Board: ###
{}
###

"""

In [4]:
"""
Adapter functions (currently valid for 4o and o1)
"""
def tictactoe_to_llm_adapter(prompt, symbol, move_number, current_situation, current_board):
    """
    Input: 
    - symbol: 'X' or 'O'
    - move_number: integer
    - current_situation: string with the following format: "Move {move_number}: {symbol} to play. Cell chosen: {cell_number}\n..
    - current_board: visual representation of the board as generated by symbolic_tic_tac_toe.py
    ###
    {board}
    ###
    Output: prompt formatted with the input values
    """
    return prompt.format(symbol, move_number, current_situation, current_board)

In [5]:
"""
Conversation builders
"""
TICTACTOE_4O_CONVERSATION = [
    {"role": "system", "content": "You are playing tic-tac-toe. Your goal is to win the game"},
]  #  conversation will be updated at each move

In [6]:
"""
Response schemas (currently valid for 4o)
"""
class TicTacToeOutputSchema(BaseModel):
    reasoning: str
    cell_number: int

In [13]:
"""
Response checkers (currently necessary for o1)
"""
def tictactoe_checking_schema(response):
    """
    Boolean function that checks if the response provided by a LLM without the response format feature adheres to the required schema (number from 1 to 9).
    Includes a pre-processing step where everything which is not a number is removed.
    Such a pre-processing step has been derived from the observation that the LLM sometimes includes additional information in the response.
    
    If the check is successful, the function returns the response in the correct format.
    Otherwise, it returns False.

    Input: string response provided by the LLM
    """
    match = re.search(r'-?\d+', response)  #  pre-processing step
    if not match:
        return False
    response = match.group(0)
    
    try:
        response = int(response)
        if response < 1 or response > 9:
            return False
    except:
        return False
         
    return response

In [8]:
tictactoe_to_llm_adapter(TICTACTOE_PROMPT_BASE_4O, 'X', 1, 'Move 1: X to play. Cell chosen: 1', '1 2 3\n4 5 6\n7 8 9')


'\nYou are playing tic-tac-toe. Your goal is to win the game. Your symbol is X. We are currently on move number 1. Below, you’ll see the current situation and a representation of the board. ou need to specify the number of the cell where you want to place your symbol. The cells are numbered from top to bottom, left to right: the first row has 1, 2, 3; the second row has 4, 5, 6; and the third row has 7, 8, 9. In your response, include a string with the reasoning you used to find the solution and the number corresponding to the cell where you want to place your symbol, like in the format below.\n\nCurrent situation: ###\nMove 1: X to play. Cell chosen: 1\n###\n\nBoard: ###\n1 2 3\n4 5 6\n7 8 9\n###\n\nDesired format: ###\nReasoning: ....\n\nCell Number: \n\n###\n'

In [16]:
sys.path.append(os.path.abspath("C:/Users/paolo/Desktop/Symbolic-AI-Vs-LLMs"))

import utils

In [17]:
client = utils.get_openai_client()


In [18]:
utils.interrogate_4o(client, "mini", TICTACTOE_4O_CONVERSATION, TicTacToeOutputSchema)

TicTacToeOutputSchema(reasoning='To win the game, I should place my marker in cell 1. This will create a winning line with cells 2 and 3, as they are already occupied by my marker.', cell_number=1)

In [ ]:
def update_game_history(current_history, symbol, move_number, cell_number):
    """
    Input: current_history, symbol, move_number, cell_number

    Updates the game history (the one that will end in the prompt) with the last move.
    """
    current_history += f"Move {move_number}: {symbol} to play. Cell chosen: {cell_number}\n"


def update_conversation(conversation, response):
    """
    Input: conversation, symbol, move_number, cell_number

    Updates the conversation between the LLM and the user.
    """
    conversation.append({"role": "assistant", "content": response})





#  4O parte da un contesto con il system prompt. A ogni step, la funzione rappresentazione visiva crea una nuova rappresentazione. A ogni step, 